In [2]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import yfinance as yf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi import NewsApiClient

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
from fuzzywuzzy import process
import requests

In [4]:
# Retrieve the News API key
news_api = os.getenv("NEWS_API_KEY")

# Create the newsapi client
newsapi = NewsApiClient(api_key=news_api)

In [5]:
# Retrieve the NYtimes API key
nyt_api = os.getenv("NYT_SECRET_KEY")

In [8]:
company_request = 'microsoft'

In [76]:
def getCompany(text):
    r = requests.get('https://api.iextrading.com/1.0/ref-data/symbols')
    stockList = r.json()
    return process.extractOne(text, stockList)[0]

In [77]:
company = getCompany(company_request)

In [78]:
symbol = company['symbol']
symbol

'MSFT'

In [79]:
company_name = company['name']

In [80]:
company_name

'MICROSOFT CORP'

In [81]:
data = yf.download(symbol, parse_dates=True, infer_datetime_format=True)

[*********************100%***********************]  1 of 1 downloaded


In [82]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.09,0.10,0.09,0.10,0.06,1031788800
1986-03-14,0.10,0.10,0.10,0.10,0.06,308160000
1986-03-17,0.10,0.10,0.10,0.10,0.07,133171200
1986-03-18,0.10,0.10,0.10,0.10,0.06,67766400
1986-03-19,0.10,0.10,0.10,0.10,0.06,47894400
...,...,...,...,...,...,...
2019-12-03,147.49,149.43,146.65,149.31,149.31,24066000
2019-12-04,150.14,150.18,149.20,149.85,149.85,17574700
2019-12-05,150.05,150.32,149.48,149.93,149.93,17869100


In [83]:
data['Daily Return'] = data['Close'].dropna().pct_change()
data

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
1986-03-13,0.09,0.10,0.09,0.10,0.06,1031788800,NaN
1986-03-14,0.10,0.10,0.10,0.10,0.06,308160000,0.000000
1986-03-17,0.10,0.10,0.10,0.10,0.07,133171200,0.000000
1986-03-18,0.10,0.10,0.10,0.10,0.06,67766400,0.000000
1986-03-19,0.10,0.10,0.10,0.10,0.06,47894400,0.000000
...,...,...,...,...,...,...,...
2019-12-03,147.49,149.43,146.65,149.31,149.31,24066000,-0.001605
2019-12-04,150.14,150.18,149.20,149.85,149.85,17574700,0.003617
2019-12-05,150.05,150.32,149.48,149.93,149.93,17869100,0.000534


In [84]:
# Set short and long windows
short_window = 5
long_window = 21

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['fast_close'] = data['Close'].ewm(halflife=short_window).mean()
data['slow_close'] = data['Close'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['fast_close'] > data['slow_close'], 1.0, 0.0)
data['crossover_short'] = np.where(data['fast_close'] < data['slow_close'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']

data.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Return,fast_close,slow_close,crossover_long,crossover_short,crossover_signal
Date,,,,,,,,,,,,
1986-03-13,0.09,0.1,0.09,0.1,0.06,1031788800,NaN,0.1,0.1,0.0,0.0,0.0
1986-03-14,0.10,0.1,0.10,0.1,0.06,308160000,0.0,0.1,0.1,0.0,0.0,0.0
1986-03-17,0.10,0.1,0.10,0.1,0.07,133171200,0.0,0.1,0.1,0.0,0.0,0.0
1986-03-18,0.10,0.1,0.10,0.1,0.06,67766400,0.0,0.1,0.1,0.0,0.0,0.0
1986-03-19,0.10,0.1,0.10,0.1,0.06,47894400,0.0,0.1,0.1,0.0,0.0,0.0


In [18]:
# Plot the EMA of BTC/USD closing prices
#data[['Close', 'fast_close', 'slow_close']].plot(figsize=(20,10))

In [19]:
news_data = newsapi.get_everything(
    q=company_name or symbol, language="en", page_size=100
)

# Show the total number of news
news_data["totalResults"]

1126

In [20]:
#news_data['articles']

In [21]:
analyzer = SentimentIntensityAnalyzer()

In [22]:
# Define a function to get the sentiment scores
def get_sentiment_scores(text, date, source, url):
    sentiment_scores = {}

    # Sentiment scoring with VADER
    text_sentiment = analyzer.polarity_scores(text)
    sentiment_scores["date"] = date
    sentiment_scores["text"] = text
    sentiment_scores["source"] = source
    sentiment_scores["url"] = url
    sentiment_scores["compound"] = text_sentiment["compound"]
    sentiment_scores["pos"] = text_sentiment["pos"]
    sentiment_scores["neu"] = text_sentiment["neu"]
    sentiment_scores["neg"] = text_sentiment["neg"]
    if text_sentiment["compound"] >= 0.05:  # Positive
        sentiment_scores["normalized"] = 1
    elif text_sentiment["compound"] <= -0.05:  # Negative
        sentiment_scores["normalized"] = -1
    else:
        sentiment_scores["normalized"] = 0  # Neutral

    return sentiment_scores

In [23]:
# Empty list to store the DataFrame structure
sentiments_data = []

# Loop through all the news articles
for article in news_data["articles"]:
    try:
        # Get sentiment scoring using the get_sentiment_score() function
        sentiments_data.append(
            get_sentiment_scores(
                article["content"],
                article["publishedAt"][:10],
                article["source"]["name"],
                article["url"],
            )
        )

    except AttributeError:
        pass

# Create a DataFrame with the news articles' data and their sentiment scoring results
news_df = pd.DataFrame(sentiments_data)

# Sort the DataFrame rows by date
news_df = news_df.sort_values(by="date")

# Define the date column as the DataFrame's index
news_df.set_index("date", inplace=True)
news_df.head()


,text,source,url,compound,pos,neu,neg,normalized
date,,,,,,,,
2019-11-09,"On CNBC's ""Mad Money Lightning Round,"" Jim Cra...",Yahoo.com,https://finance.yahoo.com/news/cramer-weighs-n...,0.5984,0.133,0.807,0.06,1
2019-11-09,We are still in an overall bull market and man...,Yahoo.com,https://finance.yahoo.com/news/orthopediatrics...,0.8519,0.220,0.780,0.00,1
2019-11-09,"Nov 9, 2019 (Thomson StreetEvents) -- Edited T...",Yahoo.com,https://finance.yahoo.com/news/edited-transcri...,0.0000,0.000,1.000,0.00,0
2019-11-10,Saudi state oil giant Aramco will sell 0.5% of...,The Hindu,https://www.thehindu.com/news/international/sa...,0.5267,0.093,0.907,0.00,1
2019-11-11,WASHINGTON (Reuters) - Microsoft Corp said in ...,Reuters,https://www.reuters.com/article/us-usa-privacy...,0.7184,0.214,0.786,0.00,1


In [24]:
news_df.tail(20)

,text,source,url,compound,pos,neu,neg,normalized
date,,,,,,,,
2019-12-05,U.S. Federal Trade Commission (FTC) has broade...,The Hindu,https://www.thehindu.com/sci-tech/technology/a...,0.0258,0.029,0.971,0.000,0
2019-12-05,Image source: The Motley Fool.\r\nSecureWorks ...,Fool.com,https://www.fool.com/earnings/call-transcripts...,0.0000,0.088,0.823,0.088,0
2019-12-06,"Just a few short years ago, buying up cryptocu...",Lifehacker.com,https://twocents.lifehacker.com/invest-in-cryp...,-0.2023,0.054,0.875,0.070,-1
2019-12-06,We are still in an overall bull market and man...,Yahoo.com,https://finance.yahoo.com/news/did-hedge-funds...,0.8176,0.184,0.816,0.000,1
2019-12-06,"By Peter Frontini\r\nSAO PAULO, Dec 6 (Reuters...",Yahoo.com,https://finance.yahoo.com/news/brazil-court-sa...,0.0000,0.000,1.000,0.000,0
2019-12-06,We are still in an overall bull market and man...,Yahoo.com,https://finance.yahoo.com/news/hedge-funds-cas...,0.8176,0.184,0.816,0.000,1
2019-12-06,We are still in an overall bull market and man...,Yahoo.com,https://finance.yahoo.com/news/hedge-funds-lov...,0.8176,0.184,0.816,0.000,1
2019-12-06,"Amid an overall bull market, many stocks that ...",Yahoo.com,https://finance.yahoo.com/news/were-hedge-fund...,0.8176,0.196,0.804,0.000,1
2019-12-06,By Pavel Alpeyev and Takahiko HyugaSoftBank Gr...,The Times of India,https://economictimes.indiatimes.com/small-biz...,-0.3400,0.028,0.911,0.061,-1


In [25]:
avg_sent_data = (
    news_df[["compound", "normalized"]].groupby(by=news_df.index).mean()
)
avg_sent_data.head()


,compound,normalized
date,,
2019-11-09,0.483433,0.666667
2019-11-10,0.526700,1.000000
2019-11-11,0.738150,1.000000
2019-11-12,0.245633,0.666667
2019-11-13,0.525633,1.000000


In [26]:
avg_sent_data

,compound,normalized
date,,
2019-11-09,0.483433,0.666667
2019-11-10,0.526700,1.000000
2019-11-11,0.738150,1.000000
2019-11-12,0.245633,0.666667
2019-11-13,0.525633,1.000000
2019-11-14,0.495325,1.000000
2019-11-15,0.521400,0.727273
2019-11-16,0.439100,0.666667
2019-11-18,0.697633,1.000000


In [27]:
signal_data =  pd.merge(data, avg_sent_data, left_index=True, right_index=True)

In [68]:
#signal_data

In [29]:
import requests
import json

In [30]:
# Create parameterized url
request_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=" + company_name + "&api-key=" + nyt_api + "&offset=500"

# Submit request and format output
response_data = requests.get(request_url).json()
#print(json.dumps(response_data, indent=4))

# Select fact 
#response_data['text']

In [57]:
import time
response=[]
for i in range(100):
    try:
        request_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=" + company_name + "&page=" + str(i) + "&api-key=" + nyt_api
        
        response_data = requests.get(request_url).json()
        data = response_data['response']['docs']
        time.sleep(6)
        for x in data:
            response.append(x)
    except:
        pass

In [58]:
len(response)

1000

In [59]:
response2 = pd.DataFrame(response)

In [60]:
response2

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,subsection_name
0,"In a legal complaint, Amazon said the presiden...",https://www.nytimes.com/2019/12/09/technology/...,"In a legal complaint, Amazon said the presiden...",SAN FRANCISCO — Amazon said in a legal complai...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Amazon Accuses Trump of ‘Improper Pr...,"[{'name': 'subject', 'value': 'Cloud Computing...",2019-12-09T16:20:47+0000,article,Business,Technology,"{'original': 'By Kate Conger', 'person': [{'fi...",News,nyt://article/4ec915d3-c88b-52d3-9ab2-28bd1587...,861,nyt://article/4ec915d3-c88b-52d3-9ab2-28bd1587...,NaN,NaN,NaN
1,Tech companies are deploying artificial intell...,https://www.nytimes.com/2019/11/12/us/online-c...,Tech companies are deploying artificial intell...,Child sexual abuse photos and videos are among...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'How Laws Against Child Sexual Abuse ...,"[{'name': 'subject', 'value': 'Child Abuse and...",2019-11-12T08:00:06+0000,article,Investigative,U.S.,{'original': 'By Gabriel J.X. Dance and Michae...,News,nyt://article/bc591503-66e3-52d7-83e8-cfd488b2...,1171,nyt://article/bc591503-66e3-52d7-83e8-cfd488b2...,B,1,NaN
2,"Makan Delrahim, Kevin Systrom and Bill Gates w...",https://www.nytimes.com/2019/11/11/business/de...,"Makan Delrahim, Kevin Systrom and Bill Gates w...","The nation’s tech giants, once celebrated as c...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'What, if Anything, Should Be Done to...","[{'name': 'subject', 'value': 'Antitrust Laws ...",2019-11-11T21:16:06+0000,article,SpecialSections,Business Day,"{'original': 'By Steve Lohr', 'person': [{'fir...",News,nyt://article/44363cf7-3b3e-53b4-9ef4-eaef66f2...,789,nyt://article/44363cf7-3b3e-53b4-9ef4-eaef66f2...,F,4,DealBook
3,"With its new Surface, Microsoft has made progr...",https://www.nytimes.com/2019/11/05/technology/...,"With its new Surface, Microsoft has made progr...","When Apple unveiled the iPad in 2010, Steve Jo...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Microsoft Surface Pro X Review: Powe...,"[{'name': 'subject', 'value': 'Computers and t...",2019-11-05T05:01:15+0000,article,Business,Technology,"{'original': 'By Brian X. Chen', 'person': [{'...",Review,nyt://article/e3784281-328d-5c7b-a21f-f39005c6...,1210,nyt://article/e3784281-328d-5c7b-a21f-f39005c6...,B,3,Personal Tech
4,The messaging service is suing a cybersurveill...,https://www.nytimes.com/2019/11/01/technology/...,The messaging service is suing a cybersurveill...,"Each week, we review the week’s news, offering...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'The Week in Tech: WhatsApp’s Spyware...,"[{'name': 'subject', 'value': 'Computers and t...",2019-11-01T13:00:06+0000,article,Business,Technology,"{'original': 'By Jamie Condliffe', 'person': [...",News,nyt://article/d1ae5989-c0ed-504e-99b3-c16aa1e7...,1251,nyt://article/d1ae5989-c0ed-504e-99b3-c16aa1e7...,B,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,,https://www.nytimes.com/2006/04/26/technology/...,,,The New York Times,[],{'main': 'Briefing: BBC seeks 'teen brand' wit...,[],2006-04-26T05:00:00+0000,article,None,Technology,"{'original': None, 'person': [], 'organization...",News,nyt://article/12ee78bb-4a54-582e-a62f-8db4d4a3...,341,nyt://article/12ee78bb-4a54-582e-a62f-8db4d4a3...,NaN,NaN,NaN
996,"James Yost, chief information officer at Ford ...",https://www.nytimes.com/2000/09/18//IHT-big-sp...,"James Yost, chief information officer at Ford ...","James Yost, chief information officer at Ford ...",International Herald Tribune,[],{'main': 'Big Spending Days for Corporate Buye...,[],2000-09-18T05:00:00+0000,

In [61]:
#response = pd.DataFrame(response_data)

In [62]:
#response = response['response']['docs']

In [63]:
#response[0]

In [64]:
# Define a function to get the sentiment scores
def get_sentiment_scores(abstract, lead_paragraph, date, url):
    sentiment_scores = {}

    # Sentiment scoring with VADER
    text_sentiment = analyzer.polarity_scores(lead_paragraph)
    sentiment_scores["pub_date"] = date
    sentiment_scores["abstract"] = abstract
    sentiment_scores["lead_paragraph"] = lead_paragraph
#    sentiment_scores["source"] = source
    sentiment_scores["web_url"] = url
    sentiment_scores["compound"] = text_sentiment["compound"]
    sentiment_scores["pos"] = text_sentiment["pos"]
    sentiment_scores["neu"] = text_sentiment["neu"]
    sentiment_scores["neg"] = text_sentiment["neg"]
    if text_sentiment["compound"] >= 0.05:  # Positive
        sentiment_scores["normalized"] = 1
    elif text_sentiment["compound"] <= -0.05:  # Negative
        sentiment_scores["normalized"] = -1
    else:
        sentiment_scores["normalized"] = 0  # Neutral

    return sentiment_scores

In [65]:
#response

In [66]:
# Empty list to store the DataFrame structure
sentiments_data = []

# Loop through all the news articles
for article in response:
    try:
        # Get sentiment scoring using the get_sentiment_score() function
        sentiments_data.append(
            get_sentiment_scores(
                article["abstract"],
                article["lead_paragraph"],
                article["pub_date"][:10],
#                article["source"],
                article["web_url"],
            )
        )

    except AttributeError:
        pass

# Create a DataFrame with the news articles' data and their sentiment scoring results
news_df = pd.DataFrame(sentiments_data)

# Sort the DataFrame rows by date
news_df = news_df.sort_values(by="pub_date")

# Define the date column as the DataFrame's index
news_df.set_index("pub_date", inplace=True)

In [71]:
news_df

,abstract,lead_paragraph,web_url,compound,pos,neu,neg,normalized
pub_date,,,,,,,,
1982-06-23,* ACF Industries has elected Irma L. Beckley...,* ACF Industries has elected Irma L. Beckley v...,https://www.nytimes.com/1982/06/23/business/ex...,0.0000,0.000,1.000,0.000,0
1986-04-17,*3*** COMPANY REPORTS **,,https://www.nytimes.com/1986/04/17/business/mi...,0.0000,0.000,0.000,0.000,0
1986-06-20,"* Dresser Industries, Dallas, said it woul...","* Dresser Industries, Dallas, said it would fo...",https://www.nytimes.com/1986/06/20/business/co...,0.0000,0.000,1.000,0.000,0
1986-07-25,Net Income Percent,See accompanying notes for special items. (Los...,https://www.nytimes.com/1986/07/25/business/pr...,0.1027,0.193,0.643,0.164,1
1986-07-25,*3*** COMPANY REPORTS **,,https://www.nytimes.com/1986/07/25/business/mi...,0.0000,0.000,0.000,0.000,0
...,...,...,...,...,...,...,...,...
2019-12-06,Brazilian delivery app Loggi must formalize wo...,SAO PAULO — Brazilian delivery app Loggi must ...,https://www.nytimes.com/reuters/2019/12/06/tec...,0.5574,0.124,0.842,0.035,1
2019-12-07,Amazon.com Inc founder Jeff Bezos said it woul...,"SIMI VALLEY, Calif. — Amazon.com Inc founder J...",https://www.nytimes.com/reuters/2019/12/07/bus...,0.6115,0.171,0.829,0.000,1
2019-12-08,Amazon.com Inc founder Jeff Bezos said it woul...,"SIMI VALLEY, Calif. — Amazon.com Inc founder J...",https://www.nytimes.com/reuters/2019/12/08/bus...,0.6115,0.171,0.829,0.000,1


In [115]:
news_sent = news_df[['compound','normalized']]

In [116]:
news_sent

,compound,normalized
pub_date,,
1982-06-23,0.0000,0
1986-04-17,0.0000,0
1986-06-20,0.0000,0
1986-07-25,0.1027,1
1986-07-25,0.0000,0
...,...,...
2019-12-06,0.5574,1
2019-12-07,0.6115,1
2019-12-08,0.6115,1


In [74]:
news_df.shape

(1000, 8)

In [48]:
#response['response'][0]

In [124]:
signal_data =  pd.concat([data, news_sent], axis=1)

In [126]:
signal_data.drop(signal_data.index[0], inplace=True)

In [127]:
signal_data.fillna(0, inplace=True)

In [128]:
signal_data= signal_data[signal_data.Open != 0]

In [129]:
signal_data

,Open,High,Low,Close,Adj Close,Volume,Daily Return,fast_close,slow_close,crossover_long,crossover_short,crossover_signal,compound,normalized
1986-03-14,0.10,0.10,0.10,0.10,0.06,308160000.0,0.000000,0.100000,0.100000,0.0,0.0,0.0,0.0,0.0
1986-03-17,0.10,0.10,0.10,0.10,0.07,133171200.0,0.000000,0.100000,0.100000,0.0,0.0,0.0,0.0,0.0
1986-03-18,0.10,0.10,0.10,0.10,0.06,67766400.0,0.000000,0.100000,0.100000,0.0,0.0,0.0,0.0,0.0
1986-03-19,0.10,0.10,0.10,0.10,0.06,47894400.0,0.000000,0.100000,0.100000,0.0,0.0,0.0,0.0,0.0
1986-03-20,0.10,0.10,0.09,0.10,0.06,58435200.0,0.000000,0.100000,0.100000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-04,150.14,150.18,149.20,149.85,149.85,17574700.0,0.003617,149.457181,143.799050,1.0,0.0,1.0,0.0,0.0
2019-12-05,150.05,150.32,149.48,149.93,149.93,17869100.0,0.000534,149.518387,143.998111,1.0,0.0,1.0,0.0,0.0
2019-12-06,150.99,151.87,150.27,151.75,151.75,16403500.0,0.012139,149.807268,144.249801,1.0,0.0,1.0,0.0,0.0
2019-12-09,151.07,152.21,150.91,151.36,151.36,14268095.0,-0.002570,150.008268,144.480657,1.0,0.0,1.0,0.0,0.0


In [87]:
type(news_df)

pandas.core.frame.DataFrame